In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pyogrio pyarrow geopandas pandas
!pip install -q osmnx
!pip install pygris

2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 45.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 42.3 MB/s eta 0:00:00


In [2]:
from torch_geometric.nn import Node2Vec
import pickle
import sys
import geopandas as gpd
import torch
from torch import Tensor
from osmnx import graph_from_gdfs
from torch_geometric.nn import Node2Vec
import networkx as nx
from torch_geometric.data import Data
from collections import defaultdict
from torch import tensor
import torch_cluster
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
from numpy.linalg import norm
import pandas as pd

from google.colab import files
import osmnx as ox
import pygris

In [3]:
# Set date for network
date = f'2020-12-28T00:00:00Z'
ox.settings.overpass_settings = '[out:json][timeout:{timeout}][date:"' + date + '"]{maxsize}'
ox.settings.overpass_url = "https://overpass-api.de/api/interpreter"

# Get the network -- simplified this time!
ntwrk = ox.graph_from_place('City of New York', simplify=True)

nodes, edges = ox.graph_to_gdfs(ntwrk)

edges.to_file('newyork_edges_2020.gpkg')
nodes.to_file('newyork_nodes_2020.gpkg')

counties = ['New York', 'Kings', 'Bronx', 'Richmond', 'Queens']
nyc_tracts = [pygris.tracts(state='New York', county=county) \
              for county in counties]
nyc_tracts = gpd.GeoDataFrame (
    pd.concat(nyc_tracts, ignore_index=True)
)

print(nyc_tracts)

nyc_tracts.to_file('nyc_tracts.shp')

Using the default year of 2021
Using FIPS code '36' for input 'New York'
Using FIPS code '061' for input 'New York'
Using the default year of 2021
Using FIPS code '36' for input 'New York'
Using FIPS code '047' for input 'Kings'
Using the default year of 2021
Using FIPS code '36' for input 'New York'
Using FIPS code '005' for input 'Bronx'
Using the default year of 2021
Using FIPS code '36' for input 'New York'
Using FIPS code '085' for input 'Richmond'
Using the default year of 2021
Using FIPS code '36' for input 'New York'
Using FIPS code '081' for input 'Queens'
     STATEFP COUNTYFP TRACTCE        GEOID    NAME             NAMELSAD  \
0         36      061  012700  36061012700     127     Census Tract 127   
1         36      061  012800  36061012800     128     Census Tract 128   
2         36      061  013000  36061013000     130     Census Tract 130   
3         36      061  013300  36061013300     133     Census Tract 133   
4         36      061  013400  36061013400     134   

In [12]:
# list of tracts that are over waterways or otherwise not relevant to this analysis
# excluded
EXCLUDED_TRACTS = ['36005051602', '36061000100']

def from_osmnx(G, nodes_gdf, edges_gdf, group_node_attrs=None, group_edge_attrs=None):
    '''
    Converts osmnx graph or digraph to torch_geometric.data.Data objects.
    Based nearly entirely on torch.geometry.utils.convert.from_networkx,
    but with modifications for this specific project.
    Args:
      G (networkx.Graph or networx.DiGraph): A networkx graph
      nodes_gdf (GeoDataFrame): the initial gdf of nodes.
      edges_gdf (GeoDataFrame)L the initial gdf of edges.
      group_node_attrs (List[str], "all", or None): The node attributes to be
        concatenated and added to data.x (defaults to None)
      group_edge_attrs (List[str], "all", or None): The edge attributes to be
        concatenated and added to data.edge_attr. Defaults to None.
       All attributes must be numeric (woe)
    '''

    G = G.to_directed() if not nx.is_directed(G) else G

    mapping = dict(zip(G.nodes(), range(G.number_of_nodes())))
    edge_index = torch.empty((2, G.number_of_edges()), dtype=torch.long)
    for i, (src, dst) in enumerate(G.edges()):
        edge_index[0, i] = mapping[src]
        edge_index[1, i] = mapping[dst]

    data_dict = defaultdict(list)
    data_dict['edge_index'] = edge_index

    node_attrs = []
    if G.number_of_nodes() > 0:
        node_attrs = list(next(iter(G.nodes(data=True)))[-1].keys())

    edge_attrs = []
    if G.number_of_edges() > 0:
        edge_attrs = list(next(iter(G.edges(data=True)))[-1].keys())

    if group_node_attrs and not isinstance(group_node_attrs, list):
        group_node_attrs = node_attrs

    if group_edge_attrs and not isinstance(group_edge_attrs, list):
        group_edge_attrs = edge_attrs

    # Main change from the initial from_networkx function are these
    # two chunks. Instead of raising an error, they now just
    # reference my initial GDFs, since I can do that.
    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        if set(feat_dict.keys()) != set(node_attrs):
            feat_dict = {}
            print(f"Correcting dropped node values for row {i}")
            for k in node_attrs:
                feat_dict[k] = nodes_gdf.iloc[i].get(k)
        for key, value in feat_dict.items():
            data_dict[str(key)].append(value)

    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        if set(feat_dict.keys()) != set(edge_attrs):
            feat_dict = {}
            print(f"Correcting dropped edge values for row {i}")
            for k in edge_attrs:
                feat_dict[k] = edges_gdf.iloc[i].get(k)
        for key, value in feat_dict.items():
            key = f'edge_{key}' if key in node_attrs else key
            data_dict[str(key)].append(value)

    for key, value in G.graph.items():
        if key == 'node_default' or key == 'edge_default':
            continue
        key = f'graph_{key}' if key in node_attrs else key
        data_dict[str(key)] = value

    for key, value in data_dict.items():
        if isinstance(value, (tuple, list)) and isinstance(value[0], Tensor):
            data_dict[key] = torch.stack(value, dim=0)
        else:
            try:
                data_dict[key] = torch.as_tensor(value)
            except Exception:
                pass

    data = Data.from_dict(data_dict)

    if group_node_attrs:
        xs = []
        for key in group_node_attrs:
            x = data[key]
            print(x)
            x = x.view(-1, 1) if x.dim() <= 1 else x
            xs.append(x)
            del data[key]
        data.x = torch.cat(xs, dim=-1)

    if group_edge_attrs:
        xs = []
        for key in group_edge_attrs:
            key = f'edge_{key}' if key in node_attrs else key
            x = data[key]
            x = x.view(-1, 1) if x.dim() <= 1 else x
            xs.append(x)
            del data[key]
        data.edge_attr = torch.cat(xs, dim=-1)

    if data.x is not None and data.pos is not None:
        data.num_nodes = G.number_of_nodes()

    return data


def setup_model(data):
    '''
    Setup our model. Returns model, loader, optimizer, device as a tuple.
    '''
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = Node2Vec(
        data.edge_index,
        embedding_dim=128,
        walk_length=20,
        context_size=10,
        walks_per_node=10,
        num_negative_samples=1,
        p=1.0,
        q=1.0,
        sparse=False
    ).to(device)

    num_workers = 4 if sys.platform == 'linux' else 0
    loader = model.loader(batch_size=129, shuffle=True, num_workers=num_workers)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.001)

    return model, loader, optimizer, device

def read_in_graphs(edges_loc="newyork_edges_2020.gpkg",
                   nodes_loc="newyork_nodes_2020_split.gpkg"):
    '''
    Read in graph data, using assumed file names and locations, and return
    both the pytorch graph and a dataset mapping from GEOIDs to graph indices.

    Defaults are based on my github repo.
    '''

    def add_geoid_to_dict(geoid_dict, geoid, i):
        '''
        Helper function which adds a new value to a dict.
        '''
        if not geoid in geoid_dict.keys():
            geoid_dict[geoid] = []
        geoid_dict[geoid].append(i)

        return geoid_dict

    edges = gpd.read_file(edges_loc, engine='pyogrio', use_arrow=True)
    edges = edges[['u', 'v', 'key', 'geometry']]
    #edges = edges.drop_duplicates(subset=['u', 'v', 'key'])
    edges = edges.set_index(['u', 'v', 'key'])

    nodes = gpd.read_file(nodes_loc, engine='pyogrio', use_arrow=True)
    nodes = nodes.drop_duplicates(subset=['osmid', 'y', 'x'])
    nodes = nodes.set_index('osmid')

    # ensure that edges u v values (first two index levels) all appear among nodes index
    uv = set(edges.index.get_level_values(0)) | set(edges.index.get_level_values(1))

    # drop edges not in nodes
    extra = uv - set(nodes.index)
    extra_mask = edges.index.get_level_values(0).isin(extra) | edges.index.get_level_values(1).isin(extra)
    if extra:
        warnings.warn(f'Trimmed {extra_mask.sum()} edges')
    filtered_edges = edges.loc[~extra_mask]

    G = graph_from_gdfs(gdf_nodes=nodes, gdf_edges=filtered_edges)

    # We need this in pytorch's graph format
    data = from_osmnx(G, nodes, edges)

    # make dict
    # above process was order preserving
    geoid_dict = {}
    for i, val in enumerate(G.nodes):
        try:
            geoid = nodes.loc[val].get('GEOID')
        except KeyError:
            print(f'KeyError on OSMID {val}.')
        if not type(geoid) in [int, str]:
            for g in geoid:
                geoid_dict = add_geoid_to_dict(geoid_dict, g, i)
            continue

        geoid_dict = add_geoid_to_dict(geoid_dict, geoid, i)

    del data.GEOID

    return data, geoid_dict

def get_crashes_by_tract(train_test=False, power_scale=False):
    '''
    Get train and test crash counts by tract for downstream tasks.
    Returns a presplit tuple if train_test is True, and power scales
    the counts if power_scale is True.
    '''
    # Read in tracts
    tracts = gpd.read_file('nyc_tracts.shp').to_crs('EPSG:2263')
    tracts = tracts[tracts['ALAND'] != 0]
    tracts = tracts[~tracts['GEOID'].isin(EXCLUDED_TRACTS)]
    crashes = gpd.read_file('crashes.shp').to_crs('EPSG:2263')

    # Take counts
    tract_crashes = gpd.sjoin(tracts, crashes, how='left', predicate='contains')
    tract_crashes = tract_crashes[['GEOID']].value_counts().reset_index()

    if power_scale:
        tract_crashes['count'] = PowerTransformer('box-cox')\
                                    .fit_transform(tract_crashes[['count']])

    if train_test:
        X_train, X_test, y_train, y_test = \
          train_test_split(tract_crashes.GEOID, tract_crashes['count'], test_size=0.2)

        return X_train, X_test, y_train, y_test

    return tract_crashes

def make_subgraph_embedding(z, indices, n_nodes=179993):
    '''
    Given a Node2Vec embedding model and the indices of relevant
    node embeddings, return the subgraph embedding as a numpy array.
    '''
    mask = torch.zeros(n_nodes, dtype=torch.bool)

    for ind in indices:
        if ind == 179993:
            continue
        mask[ind] = True

    # Take sum of nodes in subgraph
    rel = z[mask]
    rel_sum = rel.detach().cpu().numpy().sum(axis=0)
    unit = rel_sum / norm(rel_sum)
    # Return norm of sum as subgraph vector
    return unit

def train(model, loader, optimizer, device):
    '''
    Train node2vec model
    '''
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

@torch.no_grad()
def test(model, data, geoid_dict, train_test):
    '''
    Test model accuracy against regression prediction task. Returns RMSE error.
    '''
    from sklearn.linear_model import LinearRegression

    GEOID_train, GEOID_test, y_train, y_test = train_test
    model.eval()
    z = model()

    train_geoids, train_embeds = [], []
    for geoid in GEOID_train:
        train_geoids.append(geoid)
        train_embeds.append(make_subgraph_embedding(z, geoid_dict[geoid]))

    test_geoids, test_embeds = [], []
    for geoid in GEOID_test:
        test_geoids.append(geoid)
        test_embeds.append(make_subgraph_embedding(z, geoid_dict[geoid]))

    X_test = pd.DataFrame(test_embeds, index=test_geoids)
    X_train = pd.DataFrame(train_embeds, index=train_geoids)

    reg = LinearRegression().fit(X_train, y_train)

    return reg.score(X_test, y_test)

In [5]:
### Read in
nodes = gpd.read_file('newyork_nodes_2020.gpkg')
edges = gpd.read_file('newyork_edges_2020.gpkg')
tracts = gpd.read_file('nyc_tracts.shp')

old_crs = edges.crs

### Exclude streets that cross tracts
tracts.to_crs('EPSG:2263', inplace=True)
edges.to_crs('EPSG:2263',  inplace=True)
nodes.to_crs("EPSG:2263",  inplace=True)

### A surprise tool that will help us later
edges['ID'] = [f'G{i}' for i in range(len(edges))]
ids = tracts.sjoin(edges)[['GEOID', 'ID']]
print(ids.head())

### Only keep things that appear exactly once
rel_ids = ids.copy()[~ids['ID'].duplicated(keep=False)]
edges = edges[edges['ID'].isin(rel_ids['ID'].to_list())]
edges = edges.merge(ids, on='ID').drop('ID', axis=1)

### Associate nodes with a tract
nodes = nodes.sjoin(tracts[['GEOID', 'geometry']])

tracts.to_crs(old_crs, inplace=True)
edges.to_crs(old_crs,  inplace=True)
nodes.to_crs(old_crs,  inplace=True)

### Save
nodes.to_file('newyork_nodes_2020_split.gpkg')
edges.to_file('newyork_edges_2020_split.gpkg')

         GEOID       ID
0  36061012700  G158552
0  36061012700    G6216
0  36061012700    G6218
0  36061012700    G2647
0  36061012700    G7832


In [6]:
import warnings

data, geoid_dict = read_in_graphs(nodes_loc='newyork_nodes_2020_split.gpkg',
                                  edges_loc='newyork_edges_2020.gpkg')

<ipython-input-3-6c067e5a0fc3>:169: UserWarning: Trimmed 80 edges
  warnings.warn(f'Trimmed {extra_mask.sum()} edges')
/usr/local/lib/python3.10/dist-packages/osmnx/convert.py:306: UserWarning: Discarding the `gdf_nodes` 'geometry' column, though its values differ from the coordinates in the 'x' and 'y' columns.
  _validate_node_edge_gdfs(gdf_nodes, gdf_edges)


Streaming output truncated to the last 5000 lines.
Correcting dropped node values for row 95887
Correcting dropped node values for row 95888
Correcting dropped node values for row 95889
Correcting dropped node values for row 95891
Correcting dropped node values for row 95892
Correcting dropped node values for row 95894
Correcting dropped node values for row 95895
Correcting dropped node values for row 95898
Correcting dropped node values for row 95901
Correcting dropped node values for row 95903
Correcting dropped node values for row 95917
Correcting dropped node values for row 95918
Correcting dropped node values for row 95919
Correcting dropped node values for row 95922
Correcting dropped node values for row 95923
Correcting dropped node values for row 95947
Correcting dropped node values for row 95951
Correcting dropped node values for row 95954
Correcting dropped node values for row 95955
Correcting dropped node values for row 95956
Correcting dropped node values for row 95957
Corr

In [7]:
train_test = get_crashes_by_tract(True, True)

In [14]:
model, loader, optimizer, device = setup_model(data)

# These result losses and epochs are left here for posterity, but I ran this like. Twice?
# After the actual to be used weights. Those weights achieved a loss closer to
# 0.73ish and an accuracy of about ~.11ish, using the same model settings.
# I have no clue why these weights are underperforming so.
prev_best = float('-inf')
for epoch in range(1, 101):
    loss = train(model, loader, optimizer, device)
    acc = test(model, data, geoid_dict, train_test)
    if acc > prev_best:
        prev_best = acc

    # the greater than 0 check was added a tad late
    if acc < 1/5 * prev_best and prev_best > 0:
        print('Lower the learning rate, pls')

    print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, acc: {acc:.4f}")

Lower the learning rate, pls
Epoch: 001, Loss: 7.8894, acc: -0.1040
Lower the learning rate, pls
Epoch: 002, Loss: 5.7303, acc: -0.1011
Lower the learning rate, pls
Epoch: 003, Loss: 4.6066, acc: -0.0991
Lower the learning rate, pls
Epoch: 004, Loss: 3.7902, acc: -0.0978
Lower the learning rate, pls
Epoch: 005, Loss: 3.1007, acc: -0.0974
Lower the learning rate, pls
Epoch: 006, Loss: 2.5307, acc: -0.0960
Lower the learning rate, pls
Epoch: 007, Loss: 2.0756, acc: -0.0919
Lower the learning rate, pls
Epoch: 008, Loss: 1.7147, acc: -0.0901
Lower the learning rate, pls
Epoch: 009, Loss: 1.4300, acc: -0.0876
Lower the learning rate, pls
Epoch: 010, Loss: 1.2120, acc: -0.0851
Lower the learning rate, pls
Epoch: 011, Loss: 1.0528, acc: -0.0805
Lower the learning rate, pls
Epoch: 012, Loss: 0.9430, acc: -0.0756
Lower the learning rate, pls
Epoch: 013, Loss: 0.8700, acc: -0.0680
Lower the learning rate, pls
Epoch: 014, Loss: 0.8226, acc: -0.0619
Lower the learning rate, pls
Epoch: 015, Loss: 0

We're seeing gains, so let's keep going for a few generations (the below was ran quadrice, meaning we will have a total of 350 epochs in the "biggest" model. Our highest achieved accuracy occurred ~200 epochs in, with .1691).

In [ ]:
for epoch in range(1, 50):
    loss = train(model, loader, optimizer, device)
    acc = test(model, data, geoid_dict, train_test)
    if acc > prev_best:
        prev_best = acc

    # the greater than 0 check was added a tad late
    if acc < 1/5 * prev_best and prev_best > 0:
        print('Lower the learning rate, pls')

    print(f"Epoch: {epoch:03d}, Loss: {loss:.4f}, acc: {acc:.4f}")

Epoch: 001, Loss: 0.7207, acc: 0.1440
Epoch: 002, Loss: 0.7208, acc: 0.1450
Epoch: 003, Loss: 0.7207, acc: 0.1453
Epoch: 004, Loss: 0.7207, acc: 0.1484
Epoch: 005, Loss: 0.7208, acc: 0.1486
Epoch: 006, Loss: 0.7207, acc: 0.1436
Epoch: 007, Loss: 0.7207, acc: 0.1420
Epoch: 008, Loss: 0.7207, acc: 0.1420
Epoch: 009, Loss: 0.7207, acc: 0.1437
Epoch: 010, Loss: 0.7206, acc: 0.1452
Epoch: 011, Loss: 0.7207, acc: 0.1483
Epoch: 012, Loss: 0.7206, acc: 0.1498
Epoch: 013, Loss: 0.7207, acc: 0.1483
Epoch: 014, Loss: 0.7207, acc: 0.1529
Epoch: 015, Loss: 0.7206, acc: 0.1480
Epoch: 016, Loss: 0.7206, acc: 0.1470
Epoch: 017, Loss: 0.7207, acc: 0.1501
Epoch: 018, Loss: 0.7206, acc: 0.1490
Epoch: 019, Loss: 0.7206, acc: 0.1510
Epoch: 020, Loss: 0.7206, acc: 0.1528
Epoch: 021, Loss: 0.7206, acc: 0.1592
Epoch: 022, Loss: 0.7206, acc: 0.1611
Epoch: 023, Loss: 0.7206, acc: 0.1612
Epoch: 024, Loss: 0.7206, acc: 0.1579
Epoch: 025, Loss: 0.7206, acc: 0.1544
Epoch: 026, Loss: 0.7205, acc: 0.1567
Epoch: 027, 

In [25]:
# Save for future work (or for re-loading my progress!)

torch.save(model.state_dict(), 'node2vec_no_att_250_epochs')
files.download('node2vec_no_att_250_epochs')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
# Reload to pick up where we left off!
# data, geoid_dict = read_in_graphs(nodes_loc='newyork_nodes_2020_split.gpkg',
#                                   edges_loc='newyork_edges_2020.gpkg')

# model, loader, optimizer, device = setup_model(data)
# model.load_state_dict(torch.load('node2vec_no_att(1)', weights_only=True))

tract_crashes = get_crashes_by_tract()

In [26]:
import numpy as np

# merge in spatial for easier ESDA
nyc_tracts = gpd.read_file('nyc_tracts.shp')
nyc_tracts = nyc_tracts[['GEOID', 'geometry']]
nyc_tracts['GEOID'] = nyc_tracts.GEOID.astype(object)
tract_crashes2 = nyc_tracts.merge(tract_crashes, on='GEOID', how='right')

# create subgraph vectors [our features!]
model.eval()
z = model()

geoids, embeds = [], []
for geoid in tract_crashes2.GEOID:
    geoids.append(geoid)
    embeds.append(make_subgraph_embedding(z, geoid_dict[geoid]))

embed_pd = pd.DataFrame(np.array(embeds), index=geoids).reset_index()
embed_pd = embed_pd.rename({'index':'GEOID'}, axis=1)

embed_pd.to_csv('embedding_data_250.csv', index=False)

# Uncomment if using Google Colab.
from google.colab import files
files.download('embedding_data_250.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>